<div align="center">
   <h1>ANÁLISIS EXPLORATORIO</h1>
</div>

maybe add some svd pca

top 20 words in description(karen)

top 20 bigrams in descriptiom (karen)



In [3]:
#IMPORTS

import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Matplotlib is building the font cache; this may take a moment.


In [8]:
#DATABASE CONNECTION

file_path = r"C:\Users\Sarah\Code\ProyectoABD\database\linkedindatabase.db"
conn = sqlite3.connect(file_path)

print("Opened database successfully")

Opened database successfully


In [9]:
cur = conn.cursor()

In [5]:
query = "SELECT normalized_salary FROM postings WHERE normalized_salary IS NOT NULL"
df = pd.read_sql_query(query, conn)

df.head()

,normalized_salary
0,38480.0
1,83200.0
2,55000.0
3,157500.0
4,70000.0


In [6]:
# Close connection
conn.close()